# Aplicação do Código

## Organizando CUDA

In [ ]:
!nvcc --version

: 

In [ ]:
!nvidia-smi

Wed Jun  4 14:07:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!apt-get update -y
!apt-get install -y cuda-12-4

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,741 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,739 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,984 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.launchp

In [ ]:
!ls /usr/local/ | grep cuda

cuda
cuda-12
cuda-12.4
cuda-12.5


In [ ]:
!sudo rm -rf /usr/local/cuda
!sudo ln -s /usr/local/cuda-12.4 /usr/local/cuda

In [ ]:
!export PATH=/usr/local/cuda-12.4/bin:$PATH
!export LD_LIBRARY_PATH=/usr/local/cuda-12.4/lib64:$LD_LIBRARY_PATH

In [ ]:
!echo $PATH
!echo $LD_LIBRARY_PATH

/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
/usr/lib64-nvidia


## Importando Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os
# Ajuste este caminho para onde você colocar o .zip no seu Drive
caminho_zip_no_drive = "/content/drive/MyDrive/datasets/harelick.zip"
pasta_destino_colab = "dataset_busi_drive"

if os.path.exists(caminho_zip_no_drive):
    print(f"Descompactando {caminho_zip_no_drive} para {pasta_destino_colab}...")
    with zipfile.ZipFile(caminho_zip_no_drive, 'r') as zip_ref:
        zip_ref.extractall(pasta_destino_colab)
    print(f"Dataset descompactado em '{pasta_destino_colab}'.")
    !ls {pasta_destino_colab}
    if os.path.exists(f"{pasta_destino_colab}/Dataset_BUSI_with_GT"):
        print(f"\nConteúdo de '{pasta_destino_colab}/Dataset_BUSI_with_GT':")
        !ls {pasta_destino_colab}/Dataset_BUSI_with_GT
else:
    print(f"Arquivo ZIP não encontrado em: {caminho_zip_no_drive}")

Descompactando /content/drive/MyDrive/datasets/harelick.zip para dataset_busi_drive...
Dataset descompactado em 'dataset_busi_drive'.
Dataset_BUSI_with_GT

Conteúdo de 'dataset_busi_drive/Dataset_BUSI_with_GT':
benign	malignant  normal


In [ ]:
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpm0qzz8pz".


## Código CUDA

In [ ]:
import os

# Flags para OpenCV
opencv_flags_list = !pkg-config --cflags --libs opencv4
opencv_flags_str = " ".join(opencv_flags_list)

# Flags para o compilador NVCC (incluindo supressão de warnings do OpenCV, se desejar)
os.environ['NVCC_APPEND_FLAGS'] = opencv_flags_str + " -O3 -diag-suppress 611"
# Se não quiser suprimir:
# os.environ['NVCC_APPEND_FLAGS'] = opencv_flags_str + " -O3"


print(f"NVCC_APPEND_FLAGS definidos como: {os.environ['NVCC_APPEND_FLAGS']}")

NVCC_APPEND_FLAGS definidos como: -I/usr/include/opencv4 -lopencv_stitching -lopencv_alphamat -lopencv_aruco -lopencv_barcode -lopencv_bgsegm -lopencv_bioinspired -lopencv_ccalib -lopencv_dnn_objdetect -lopencv_dnn_superres -lopencv_dpm -lopencv_face -lopencv_freetype -lopencv_fuzzy -lopencv_hdf -lopencv_hfs -lopencv_img_hash -lopencv_intensity_transform -lopencv_line_descriptor -lopencv_mcc -lopencv_quality -lopencv_rapid -lopencv_reg -lopencv_rgbd -lopencv_saliency -lopencv_shape -lopencv_stereo -lopencv_structured_light -lopencv_phase_unwrapping -lopencv_superres -lopencv_optflow -lopencv_surface_matching -lopencv_tracking -lopencv_highgui -lopencv_datasets -lopencv_text -lopencv_plot -lopencv_ml -lopencv_videostab -lopencv_videoio -lopencv_viz -lopencv_wechat_qrcode -lopencv_ximgproc -lopencv_video -lopencv_xobjdetect -lopencv_objdetect -lopencv_calib3d -lopencv_imgcodecs -lopencv_features2d -lopencv_dnn -lopencv_flann -lopencv_xphoto -lopencv_photo -lopencv_imgproc -lopencv_core  

In [ ]:
%%cuda
#include <cuda_runtime.h>
#include <cooperative_groups.h>
#include <device_launch_parameters.h>
#include <iostream>
#include <string>
#include <vector>
#include <fstream> 
#include <opencv2/opencv.hpp>
#include <iomanip> 
#include <algorithm>

// Constante para o tamanho do warp a ser usada no código do host (CPU)
// Também define o tamanho da agregação para a lógica do algoritmo.
const int HOST_DEFINED_WARP_SIZE = 32;

using namespace cooperative_groups;

// Objeto de textura global
cudaTextureObject_t texObj = 0;

// Função para salvar um std::vector<float> em um arquivo de texto
void saveVectorToFile(const std::vector<float>& data, const std::string& filename) {
    std::ofstream outFile(filename);
    if (outFile.is_open()) {
        outFile << std::fixed << std::setprecision(8); // Salva com boa precisão
        for (size_t i = 0; i < data.size(); ++i) {
            outFile << data[i] << std::endl;
        }
        outFile.close();
        std::cout << "Vetor de saída bruto salvo em: " << filename << std::endl;
    } else {
        std::cerr << "ERRO: Não foi possível abrir o arquivo para salvar: " << filename << std::endl;
    }
}

// Kernel CUDA
__global__ void haralick_texture_kernel(cudaTextureObject_t tex,
                                        float *out, int width, int height) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x >= width || y >= height) return;

    float center = tex2D<float>(tex, x, y);
    float right  = (x + 1 < width) ? tex2D<float>(tex, x + 1, y) : center;

    float val = (center + right) * 0.5f;
    for (int offset = warpSize / 2; offset > 0; offset /= 2) { // warpSize aqui é o do device
        val += __shfl_down_sync(0xffffffff, val, offset);
    }

    if ((threadIdx.y * blockDim.x + threadIdx.x) % warpSize == 0) { // warpSize aqui é o do device
        // y e x aqui são as coordenadas globais da primeira thread do warp
        int idx = (y * width + x) / warpSize; // warpSize aqui é o do device
        out[idx] = val;
    }
}

int main(int argc, char** argv) {
    std::cout << std::fixed << std::setprecision(8);

    bool use_synthetic_image = true;

    cv::Mat imgF_for_processing;
    int width;
    int height;



    if (use_synthetic_image) {
        std::cout << "--- Usando Imagem Sintética para Teste ---" << std::endl;
        int test_width = HOST_DEFINED_WARP_SIZE; // Ex: 32
        int test_height = 1;                   // Apenas 1 linha para o teste mais simples

        // Cria a imagem CV_32F
        imgF_for_processing = cv::Mat(test_height, test_width, CV_32F);

        // Preenche todos os pixels com 0.5f
        imgF_for_processing.setTo(cv::Scalar(0.5f));

        // Se quisesse um padrão diferente, poderia usar loops:
        // for (int r = 0; r < test_height; ++r) {
        //     for (int c = 0; c < test_width; ++c) {
        //         // Exemplo: um gradiente simples ou outro padrão
        //         imgF_for_processing.at<float>(r, c) = static_cast<float>(c) / (test_width > 1 ? (test_width - 1) : 1);
        //     }
        // }

        width = imgF_for_processing.cols;
        height = imgF_for_processing.rows;
        std::cout << "Imagem sintética criada. Dimensões (cols x rows): " << width << "x" << height << std::endl;

        // Imprimir alguns pixels da imagem sintética para confirmar
        std::cout << "CUDA main: Verificando pixels da imagem sintética imgF_for_processing:" << std::endl;
        int print_cols = std::min(width, 5); // Imprime até 5 colunas
        int print_rows = std::min(height, 5); // Imprime até 5 linhas
        for (int r_idx = 0; r_idx < print_rows; ++r_idx) {
            for (int c_idx = 0; c_idx < print_cols; ++c_idx) {
                 std::cout << imgF_for_processing.at<float>(r_idx, c_idx) << " ";
            }
            std::cout << std::endl;
        }

    } else { // Usar imagem do arquivo (seu código original)
        std::cout << "--- Usando Imagem do Arquivo ---" << std::endl;
        std::string caminho_da_imagem = "/content/dataset_busi_drive/Dataset_BUSI_with_GT/benign/benign (1).png"; // SEU CAMINHO
        if (argc > 1 && argv[1] != nullptr) {
            caminho_da_imagem = argv[1];
        }
        std::cout << "Carregando imagem do arquivo: " << caminho_da_imagem << std::endl;

        cv::Mat img_orig = cv::imread(caminho_da_imagem, cv::IMREAD_GRAYSCALE);
        if (img_orig.empty()) {
            std::cerr << "Erro fatal: Falha ao carregar a imagem: " << caminho_da_imagem << std::endl;
            return EXIT_FAILURE;
        }
        std::cout << "Imagem '" << caminho_da_imagem << "' carregada. Dimensões (cols x rows): " << img_orig.cols << "x" << img_orig.rows << std::endl;
        if(img_orig.cols == 0 || img_orig.rows == 0) {
            std::cerr << "Erro: Imagem carregada tem dimensões zero." << std::endl;
            return EXIT_FAILURE;
        }

        width  = img_orig.cols;
        height = img_orig.rows;
        img_orig.convertTo(imgF_for_processing, CV_32F, 1.0 / 255.0); // Converte para imgF_for_processing
    }

    // VETOR CUDA: Verificando pixels de imgF na linha 235 (antes de copiar para GPU):
    // Objetivo: Confirmar que os dados em imgF no host são os esperados.
    std::cout << "CUDA main: Verificando pixels de imgF na linha 235 (antes de copiar para GPU):" << std::endl;
    if (height > 235 && width > 260) { // Verifica se a imagem é grande o suficiente para a amostragem
        const float* row_ptr_debug = imgF_for_processing.ptr<float>(235); // Pega o ponteiro para a linha 235
        for (int j = 256; j <= std::min(260, width - 1); ++j) { // Itera pelas colunas 256 a 260 (ou até o limite da imagem)
            std::cout << "imgF(235, " << j << "): " << row_ptr_debug[j] << " ";
        }
        std::cout << std::endl;
    } else {
        std::cout << "Imagem pequena demais para a amostragem de pixels na linha 235, cols 256-260." << std::endl;
    }
    // FIM DA VERIFICAÇÃO DOS PIXELS DE imgF NO HOST



    // --- Alocação e Configuração CUDA ---
    cudaChannelFormatDesc channelDesc = cudaCreateChannelDesc<float>();
    cudaArray_t cuArray = nullptr;
    cudaError_t err = cudaMallocArray(&cuArray, &channelDesc, width, height);
    if (err != cudaSuccess) {
        std::cerr << "Erro cudaMallocArray: " << cudaGetErrorString(err) << std::endl;
        return EXIT_FAILURE;
    }

    size_t srcPitch = width * sizeof(float);
    err = cudaMemcpy2DToArray(cuArray, 0, 0, imgF_for_processing.ptr<float>(), srcPitch, width * sizeof(float), height, cudaMemcpyHostToDevice);
    if (err != cudaSuccess) {
        std::cerr << "Erro cudaMemcpy2DToArray: " << cudaGetErrorString(err) << std::endl;
        cudaFreeArray(cuArray); return EXIT_FAILURE;
    }

    cudaResourceDesc resDesc = {};
    resDesc.resType = cudaResourceTypeArray;
    resDesc.res.array.array = cuArray;
    cudaTextureDesc texDesc = {};
    texDesc.addressMode[0] = cudaAddressModeClamp; texDesc.addressMode[1] = cudaAddressModeClamp;
    texDesc.filterMode = cudaFilterModePoint; texDesc.readMode = cudaReadModeElementType;
    texDesc.normalizedCoords = 0;

    // texObj é global, não precisa ser redeclarado.
    err = cudaCreateTextureObject(&texObj, &resDesc, &texDesc, nullptr);
    if (err != cudaSuccess) {
        std::cerr << "Erro cudaCreateTextureObject: " << cudaGetErrorString(err) << std::endl;
        cudaFreeArray(cuArray); return EXIT_FAILURE;
    }

    // Tamanho do buffer de saída 1D. Corresponde ao máximo índice que o kernel pode escrever.
    int outSize = (width * height) / HOST_DEFINED_WARP_SIZE;
    if (outSize <= 0) {
        std::cerr << "Erro: outSize calculado é <= 0. Imagem (width=" << width << ", height=" << height
                  << ") muito pequena para o HOST_DEFINED_WARP_SIZE (" << HOST_DEFINED_WARP_SIZE << ")." << std::endl;
        cudaDestroyTextureObject(texObj); cudaFreeArray(cuArray); return EXIT_FAILURE;
    }
    float *d_out = nullptr;
    err = cudaMalloc(&d_out, outSize * sizeof(float));
    if (err != cudaSuccess) {
        std::cerr << "Erro cudaMalloc para d_out: " << cudaGetErrorString(err) << std::endl;
        cudaDestroyTextureObject(texObj); cudaFreeArray(cuArray); return EXIT_FAILURE;
    }

    dim3 block(32, 8);
    dim3 grid( (width + block.x - 1) / block.x, (height + block.y - 1) / block.y);
    std::cout << "Grid CUDA: " << grid.x << "x" << grid.y << ", Block: " << block.x << "x" << block.y << std::endl;

    // --- Medição de Tempo CUDA ---
    cudaEvent_t start_event, stop_event;
    cudaEventCreate(&start_event);
    cudaEventCreate(&stop_event);

    std::cout << "Iniciando processamento CUDA..." << std::endl;
    cudaEventRecord(start_event);

    haralick_texture_kernel<<<grid, block>>>(texObj, d_out, width, height);

    // Verificar erros de lançamento do kernel
    err = cudaGetLastError();
    if (err != cudaSuccess) {
        std::cerr << "Erro no lançamento do kernel CUDA ou kernel anterior: " << cudaGetErrorString(err) << std::endl;
        cudaDestroyTextureObject(texObj); cudaFreeArray(cuArray); cudaFree(d_out);
        cudaEventDestroy(start_event); cudaEventDestroy(stop_event);
        return EXIT_FAILURE;
    }

    cudaEventRecord(stop_event);
    cudaEventSynchronize(stop_event);

    float milliseconds_cuda = 0;
    cudaEventElapsedTime(&milliseconds_cuda, start_event, stop_event);
    std::cout << ">>> Tempo de execução do Kernel CUDA: " << milliseconds_cuda << " ms <<<" << std::endl;
    // --- Fim Medição CUDA ---

    std::vector<float> h_out_cuda(outSize); // Usa o outSize maior para pegar todos os dados do kernel
    err = cudaMemcpy(h_out_cuda.data(), d_out, outSize * sizeof(float), cudaMemcpyDeviceToHost);
     if (err != cudaSuccess) {
        std::cerr << "Erro cudaMemcpy DtoH: " << cudaGetErrorString(err) << std::endl;
        // Cleanup...
        cudaDestroyTextureObject(texObj); cudaFreeArray(cuArray); cudaFree(d_out);
        cudaEventDestroy(start_event); cudaEventDestroy(stop_event);
        return EXIT_FAILURE;
    }

    // Salva o vetor bruto h_out_cuda (que tem 'outSize' elementos)
    saveVectorToFile(h_out_cuda, "cuda_output_raw.txt");

    // --- Salvar Imagem de Saída CUDA ---
    // A imagem final terá 'height' linhas e 'newW_cuda' colunas.
    // O número de elementos para a imagem é height * newW_cuda.
    int newW_cuda = width / HOST_DEFINED_WARP_SIZE;
    if (newW_cuda <= 0 && width > 0) newW_cuda = 1;
    else if (width == 0) newW_cuda = 0;

    // Número de elementos que realmente formarão a imagem 2D
    int elements_for_image = height * newW_cuda;

    if (newW_cuda > 0 && height > 0 && h_out_cuda.size() >= elements_for_image && elements_for_image > 0) {
        cv::Mat outImg_cuda(height, newW_cuda, CV_32F);
        for (int r = 0; r < height; ++r) {
            for (int c = 0; c < newW_cuda; ++c) {
                int idx = r * newW_cuda + c; // Índice para a imagem 2D e para o vetor 1D relevante
                // Não precisa mais de 'if (idx < h_out_cuda.size())' aqui se elements_for_image é usado
                // para garantir que h_out_cuda é grande o suficiente, mas não custa manter por segurança
                // se h_out_cuda fosse menor que elements_for_image por algum erro.
                if (idx < h_out_cuda.size()){
                     outImg_cuda.at<float>(r, c) = h_out_cuda[idx];
                } else {
                     outImg_cuda.at<float>(r, c) = 0.0f; // Fallback, não deveria acontecer
                }
            }
        }
        cv::Mat outImg8U_cuda;
        cv::normalize(outImg_cuda, outImg_cuda, 0, 255, cv::NORM_MINMAX);
        outImg_cuda.convertTo(outImg8U_cuda, CV_8U);
        std::cout << "Dimensões da imagem de saída CUDA (outImg_cuda): "
                  << outImg_cuda.cols << "x" << outImg_cuda.rows << std::endl;
        if (cv::imwrite("haralick_out_cuda.png", outImg8U_cuda)) {
            std::cout << "Imagem CUDA salva em haralick_out_cuda.png" << std::endl;
        } else {
            std::cerr << "Falha ao salvar haralick_out_cuda.png" << std::endl;
        }
    } else {
        std::cout << "Imagem de saída CUDA não gerada devido a dimensões inválidas ou tamanho de h_out_cuda insuficiente." << std::endl;
        std::cout << "(newW_cuda=" << newW_cuda << ", height=" << height
                  << ", h_out_cuda.size()=" << h_out_cuda.size()
                  << ", elements_for_image=" << elements_for_image << ")" << std::endl;
    }

    // Cleanup CUDA
    cudaDestroyTextureObject(texObj);
    cudaFreeArray(cuArray);
    cudaFree(d_out);
    cudaEventDestroy(start_event);
    cudaEventDestroy(stop_event);

    std::cout << "--- Fim da execução CUDA ---" << std::endl;
    return EXIT_SUCCESS;
}

--- Usando Imagem Sintética para Teste ---
Imagem sintética criada. Dimensões (cols x rows): 32x1
CUDA main: Verificando pixels da imagem sintética imgF_for_processing:
0.50000000 0.50000000 0.50000000 0.50000000 0.50000000 
CUDA main: Verificando pixels de imgF na linha 235 (antes de copiar para GPU):
Imagem pequena demais para a amostragem de pixels na linha 235, cols 256-260.
Grid CUDA: 1x1, Block: 32x8
Iniciando processamento CUDA...
>>> Tempo de execução do Kernel CUDA: 112.86275482 ms <<<
Vetor de saída bruto salvo em: cuda_output_raw.txt
Dimensões da imagem de saída CUDA (outImg_cuda): 1x1
Imagem CUDA salva em haralick_out_cuda.png
--- Fim da execução CUDA ---



## Código OpenMP

In [ ]:
%%writefile haralick_texture_openmp.cpp
#include <iostream>
#include <fstream>
#include <string>
#include <vector>
#include <chrono>    // Para medição de tempo
#include <cmath>     // Para std::abs (na verificação)
#include <iomanip>   // Para std::fixed e std::setprecision
#include <omp.h>       // Para OpenMP
#include <opencv2/opencv.hpp>

// Constante para o tamanho da agregação, análogo ao warpSize na lógica CUDA
const int AGGREGATION_SIZE = 32;

// Função para salvar um std::vector<float> em um arquivo de texto
void saveVectorToFile(const std::vector<float>& data, const std::string& filename) {
    std::ofstream outFile(filename);
    if (outFile.is_open()) {
        // Define a precisão para garantir que os floats sejam salvos com detalhes suficientes
        outFile << std::fixed << std::setprecision(8); // Ajuste a precisão conforme necessário
        for (size_t i = 0; i < data.size(); ++i) {
            outFile << data[i] << std::endl;
        }
        outFile.close();
        std::cout << "Vetor de saída salvo em: " << filename << std::endl;
    } else {
        std::cerr << "ERRO: Não foi possível abrir o arquivo para salvar: " << filename << std::endl;
    }
}

// Função OpenMP que replica a lógica do kernel CUDA
void haralick_texture_openmp(const cv::Mat& imgF, // Input CV_32F image
                             std::vector<float>& h_out_omp,
                             int aggregation_size) {
    int height = imgF.rows;
    int width = imgF.cols;

    if (height == 0 || width == 0) {
        h_out_omp.clear();
        return;
    }

    int newW = width / aggregation_size;
    if (newW <= 0 && width > 0) newW = 1;
    else if (width == 0) newW = 0;

    if (newW == 0 && height > 0) {
        h_out_omp.clear();
        return;
    }
    h_out_omp.assign(height * newW, 0.0f);

    // Opcional: Defina o número de threads OpenMP
    // omp_set_num_threads(omp_get_max_threads());
    // Ou use a variável de ambiente OMP_NUM_THREADS antes de rodar o executável

    #pragma omp parallel for schedule(dynamic)
    for (int r = 0; r < height; ++r) { // height aqui é imgF.rows = 471
    const float* row_ptr = imgF.ptr<float>(r);
    for (int c_out = 0; c_out < newW; ++c_out) { // newW aqui é (imgF.cols / aggregation_size) = 562 / 32 = 17
        float current_segment_sum = 0.0f;
        int start_x_original = c_out * aggregation_size;

        // Para depurar um pixel de saída específico
        bool debug_this_pixel = (r == 235 && c_out == 8);

        if (debug_this_pixel) {
            std::cout << "\n--- Depurando r=" << r << ", c_out=" << c_out
                      << " (indice 1D = " << (r * newW + c_out) << ") ---" << std::endl;
            std::cout << "imgF.rows (height_func): " << height
                      << ", imgF.cols (width_func): " << width // width e height aqui são os da função
                      << ", aggregation_size: " << aggregation_size
                      << ", newW (calculado na func): " << newW << std::endl;
            std::cout << "start_x_original: " << start_x_original << std::endl;
            std::cout << "--- Loop Interno (i de 0 a " << aggregation_size - 1 << ") ---" << std::endl;
        }

        for (int i = 0; i < aggregation_size; ++i) {
            int x_original = start_x_original + i;

            if (x_original >= width) { // width aqui é imgF.cols = 562
                if (debug_this_pixel) {
                     std::cout << "i=" << i << ", x_original=" << x_original
                               << " SAINDO DO LIMITE (width_func=" << width << "). Parando loop interno." << std::endl;
                }
                break;
            }

            float center_val = row_ptr[x_original];
            float right_val = (x_original + 1 < width) ? // width aqui é imgF.cols = 562
                              row_ptr[x_original + 1] :
                              center_val;

            float term = (center_val + right_val) * 0.5f;
            current_segment_sum += term;

            if (debug_this_pixel) {
                std::cout << "i=" << std::setw(2) << i
                          << ", x_orig=" << std::setw(3) << x_original
                          << ", center=" << std::setw(9) << center_val
                          << ", right=" << std::setw(9) << right_val
                          << ", term=" << std::setw(9) << term
                          << ", current_sum=" << std::setw(12) << current_segment_sum << std::endl;
            }
        }
        // A atribuição a h_out_omp acontece aqui, após o loop interno
        if (r * newW + c_out < h_out_omp.size()) {
             h_out_omp[r * newW + c_out] = current_segment_sum;
             if (debug_this_pixel) {
                 std::cout << "--- VALOR FINAL para h_out_omp[" << (r * newW + c_out) << "]: "
                           << current_segment_sum << " ---" << std::endl;
             }
        }
    }
}
}

int main(int argc, char** argv) {
    std::cout << std::fixed << std::setprecision(3); // Configura cout para 3 casas decimais para tempo

    // !!! IMPORTANTE: SUBSTITUA PELO CAMINHO CORRETO DA SUA IMAGEM !!!
    //std::string caminho_da_imagem ="/content/dataset_busi_drive/Dataset_BUSI_with_GT/benign/benign1.png";
    // Exemplo para Colab se descompactado direto em /content"
    std::string caminho_da_imagem = "/content/dataset_busi/Dataset_BUSI_with_GT/benign/benign (1).png";

    if (argc > 1 && argv[1] != nullptr) {
        caminho_da_imagem = argv[1];
        std::cout << "Usando imagem fornecida como argumento: " << caminho_da_imagem << std::endl;
    } else {
        std::cout << "Nenhum argumento de imagem fornecido, usando imagem padrão: " << caminho_da_imagem << std::endl;
    }

    cv::Mat img_orig = cv::imread(caminho_da_imagem, cv::IMREAD_GRAYSCALE);
    if (img_orig.empty()) {
        std::cerr << "Erro fatal: Falha ao carregar a imagem: " << caminho_da_imagem << std::endl;
        return EXIT_FAILURE;
    }
    std::cout << "Imagem '" << caminho_da_imagem << "' carregada. Dimensões: " << img_orig.cols << "x" << img_orig.rows << std::endl;

    if(img_orig.cols == 0 || img_orig.rows == 0) {
        std::cerr << "Erro: Imagem carregada tem dimensões zero." << std::endl;
        return EXIT_FAILURE;
    }

  int width  = img_orig.cols; // Largura real da imagem de entrada
  int height = img_orig.rows; // Altura real da imagem de entrada

    cv::Mat imgF;
    img_orig.convertTo(imgF, CV_32F, 1.0 / 255.0);

    std::vector<float> h_out_omp;

    // --- Medição de Tempo OpenMP ---
    std::cout << "Iniciando processamento OpenMP..." << std::endl;
    auto start_omp = std::chrono::high_resolution_clock::now();
    std::cout << "Verificando alguns pixels de imgF antes de chamar OpenMP:" << std::endl;
    for (int i = 0; i < std::min(5, imgF.rows); ++i) {
    for (int j = 0; j < std::min(5, imgF.cols); ++j) {
        std::cout << imgF.at<float>(i, j) << " ";
    }
    std::cout << std::endl;
    }
    haralick_texture_openmp(imgF, h_out_omp, AGGREGATION_SIZE);

    auto stop_omp = std::chrono::high_resolution_clock::now();
    auto duration_omp = std::chrono::duration_cast<std::chrono::milliseconds>(stop_omp - start_omp);
    std::cout << ">>> Tempo de execução OpenMP: " << duration_omp.count() << " ms <<<" << std::endl;
    // --- Fim Medição OpenMP ---

    // ADICIONE ESTA LINHA PARA SALVAR O VETOR OPENMP:
    saveVectorToFile(h_out_omp, "omp_output_raw.txt");

    // --- Salvar Imagem de Saída OpenMP ---
    int newW_omp = width / AGGREGATION_SIZE;
    if (newW_omp <= 0 && width > 0) newW_omp = 1;
    else if (width == 0) newW_omp = 0;

    if (newW_omp > 0 && height > 0 && !h_out_omp.empty()) {
        cv::Mat outImg_omp(height, newW_omp, CV_32F);
        for (int r = 0; r < height; ++r) {
            for (int c = 0; c < newW_omp; ++c) {
                int idx = r * newW_omp + c;
                if (idx < h_out_omp.size()) outImg_omp.at<float>(r, c) = h_out_omp[idx];
                else outImg_omp.at<float>(r, c) = 0.0f;
            }
        }
        cv::Mat outImg8U_omp;
        cv::normalize(outImg_omp, outImg_omp, 0, 255, cv::NORM_MINMAX);
        outImg_omp.convertTo(outImg8U_omp, CV_8U);
        std::cout << "Dimensões da imagem de saída OpenMP (outImg_omp): "
          << outImg_omp.cols << "x" << outImg_omp.rows << std::endl;
        if (cv::imwrite("haralick_out_omp.png", outImg8U_omp)) {
            std::cout << "Imagem OpenMP salva em haralick_out_omp.png" << std::endl;
        } else {
            std::cerr << "Falha ao salvar haralick_out_omp.png" << std::endl;
        }
    }

    std::cout << "--- Fim da execução OpenMP ---" << std::endl;
    return EXIT_SUCCESS;
}

In [ ]:
!g++ haralick_texture_openmp.cpp -o haralick_omp -O3 -fopenmp $(pkg-config --cflags --libs opencv4)
!./haralick_omp '/content/dataset_busi_drive/Dataset_BUSI_with_GT/benign/benign (1).png'

In [ ]:
!diff cuda_output_raw.txt omp_output_raw.txt

In [ ]:
import numpy as np
import sys # Para aumentar o limite de impressão do numpy

# Aumenta o limite de impressão para ver mais elementos se necessário
# np.set_printoptions(threshold=sys.maxsize) # Use com cautela para vetores muito grandes

def load_vector_from_file(filename):
    try:
        # Tenta carregar como float32 para corresponder ao C++ float
        data = np.loadtxt(filename, dtype=np.float32)
        return data
    except Exception as e:
        print(f"Erro ao carregar {filename}: {e}")
        return None

vec_cuda_full = load_vector_from_file("cuda_output_raw.txt")
vec_omp = load_vector_from_file("omp_output_raw.txt")

if vec_cuda_full is not None and vec_omp is not None:
    print(f"Tamanho original CUDA: {vec_cuda_full.shape}, Tamanho original OMP: {vec_omp.shape}")

    size_omp = vec_omp.shape[0]
    size_cuda_full = vec_cuda_full.shape[0]

    vec_cuda_to_compare = None

    if size_cuda_full == size_omp:
        print("Vetores CUDA e OMP têm o mesmo tamanho. Comparando diretamente.")
        vec_cuda_to_compare = vec_cuda_full
    elif size_cuda_full > size_omp:
        print(f"Vetor CUDA ({size_cuda_full}) é maior que OMP ({size_omp}).")
        print(f"Truncando vetor CUDA para os primeiros {size_omp} elementos para comparação.")
        vec_cuda_to_compare = vec_cuda_full[:size_omp]
    elif size_omp > size_cuda_full: # Não deveria acontecer com o código atual
        print(f"Vetor OMP ({size_omp}) é maior que CUDA ({size_cuda_full}).")
        print(f"Truncando vetor OMP para os primeiros {size_cuda_full} elementos para comparação.")
        vec_cuda_to_compare = vec_cuda_full # Compara o CUDA inteiro
        vec_omp = vec_omp[:size_cuda_full] # Trunca o OMP
    else:
        print("Erro inesperado na lógica de tamanho dos vetores.")
        # Define para None para evitar erro na próxima verificação
        vec_cuda_to_compare = None
        vec_omp = None


    if vec_cuda_to_compare is not None and vec_omp is not None and \
       vec_cuda_to_compare.shape == vec_omp.shape and vec_omp.shape[0] > 0:

        print(f"Comparando vetores com tamanho efetivo: {vec_omp.shape[0]}")

        diff = np.abs(vec_cuda_to_compare - vec_omp)

        print("\n--- Estatísticas da Diferença ---")
        print(f"Diferença Máxima: {np.max(diff):.8f}")
        print(f"Diferença Média: {np.mean(diff):.8f}")
        print(f"Soma Total da Diferença Absoluta: {np.sum(diff):.8f}")

        # Contar quantos elementos são significativamente diferentes
        # Ajuste a tolerância (atol) conforme necessário. 1e-4 ou 1e-5 é comum.
        tolerance = 1e-4
        num_diff_elements = np.sum(diff > tolerance)
        print(f"Número de elementos com diferença > {tolerance}: {num_diff_elements} de {vec_omp.shape[0]}")

        if np.allclose(vec_cuda_to_compare, vec_omp, atol=tolerance):
            print("Os vetores são NUMERICAMENTE PRÓXIMOS (dentro da tolerância)!")
        else:
            print("Os vetores TÊM DIFERENÇAS NUMÉRICAS SIGNIFICATIVAS (além da tolerância).")

        print("\n--- Amostra das Diferenças (CUDA - OMP) ---")
        print("Índice | Val CUDA    | Val OMP     | Diferença")
        print("-------|-------------|-------------|-----------")
        sample_size = min(15, vec_omp.shape[0]) # Mostrar até 15 amostras
        for i in range(sample_size):
            print(f"{i:<6} | {vec_cuda_to_compare[i]:<11.6f} | {vec_omp[i]:<11.6f} | {diff[i]:<9.6f}")
        if vec_omp.shape[0] > sample_size * 2 : # Mostrar algumas do meio se for grande
            print("...")
            mid_start = vec_omp.shape[0] // 2
            for i in range(mid_start, min(mid_start + 5, vec_omp.shape[0])):
                 print(f"{i:<6} | {vec_cuda_to_compare[i]:<11.6f} | {vec_omp[i]:<11.6f} | {diff[i]:<9.6f}")
        if vec_omp.shape[0] > sample_size : # Mostrar algumas do final
            print("...")
            end_start = max(sample_size, vec_omp.shape[0] - 5) # Garante que não tentamos mostrar amostras já mostradas
            for i in range(end_start, vec_omp.shape[0]):
                 print(f"{i:<6} | {vec_cuda_to_compare[i]:<11.6f} | {vec_omp[i]:<11.6f} | {diff[i]:<9.6f}")

    elif vec_cuda_to_compare is None or vec_omp is None :
        print("Não foi possível realizar a comparação porque um dos vetores não foi carregado ou definido.")
    else:
        print("Não foi possível realizar a comparação devido a tamanhos finais inconsistentes ou vetores vazios.")
        print(f"Forma final CUDA para comparar: {vec_cuda_to_compare.shape if vec_cuda_to_compare is not None else 'N/A'}")
        print(f"Forma final OMP para comparar: {vec_omp.shape if vec_omp is not None else 'N/A'}")

else:
    print("Não foi possível carregar um ou ambos os arquivos de vetor (`cuda_output_raw.txt`, `omp_output_raw.txt`).")